<a href="https://colab.research.google.com/github/joieaurum/RecurrentNeuralNetworks/blob/main/RNN_for_Sequences_Sentiment_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
*Natural language Processing  
*Look at this in a very simple manner either positive or negative and since its that then its a binary classification problem  
-Sequence to sequence-encoder RNN-time series, text in sentences  
-Recurrent because the network contains loops  
-output of a given layer is an input to the same layer time step  
-Time step ->time series, next word in a sequence of words text sequence  
-RNN is attractive cause of the loops, which help them manage relationships

-In this example i will use the Long Short Term Memory(LSTM) layer->this is  
-what makes the network recurrent
it is optimized to learning sequences like predicting text input-chatbot,interlanguage translation,automated video closed caption'Speech recognition',speech synthesis

## Loading the data

In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
number_of_words=10000

In [ ]:
import numpy as np

### modifying the default parameters of np.load

In [ ]:
# save np.load
np_load_old = np.load


In [ ]:
# modify the default parameters of np.load
np.load=lambda *a,**k:np_load_old(*a,allow_pickle=True)

In [ ]:
# call load_data with allow_pickle implicitly set to true
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=number_of_words)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
# restore np.load for future normal usage
np.load = np_load_old

## EDA

In [ ]:
X_train.shape

(25000,)

In [ ]:
y_train.shape

(25000,)

In [ ]:
X_test.shape

(25000,)

In [ ]:
y_test.shape

(25000,)

-The arrays y_train and y_test -1 D,1s(positive) and 0s(negative)  
-The X_test and X_train are a list of integers that represent the viewers comment

In [ ]:
%pprint

Pretty printing has been turned OFF


In [ ]:
X_train[2]

[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5974, 54, 61, 369, 13, 71, 149, 14, 22, 112, 4, 2401, 311, 12, 16, 3711, 33, 75, 43, 1829, 296, 4, 86, 320, 35, 534, 19, 263, 4821, 1301, 4, 1873, 33, 89, 78, 12, 66, 16, 4, 360, 7, 4, 58, 316, 334, 11, 4, 1716, 43, 645, 662, 8, 257, 85, 1200, 42, 1228, 2578, 83, 68, 3912, 15, 36, 165, 1539, 278, 36, 69, 2, 780, 8, 106, 14, 6905, 1338, 18, 6, 22, 12, 215, 28, 610, 40, 6, 87, 326, 23, 2300, 21, 23, 22, 12, 272, 40, 57, 31, 11, 4, 22, 47, 6, 2307, 51, 9, 170, 23, 595, 116, 595, 1352, 13, 191, 79, 638, 89, 2, 14, 9, 8, 106, 607, 624, 35, 534, 6, 227, 7, 129, 113]

In [ ]:
y_train[2]

0

Review the encodings  
movies are numerically encoded,,text-word which each number corresponds to.  
Keras IMDB dictionary maps words to their indices  
Each word's value is its frequency ranking among the words in the dataset  
1->is the most frequent word  
offset by 3- training and trsting samples = 4 -0,1,2-reserved
0-padding -dimensions 0000000 i do not like it  
1-Start of the sequence - a token that keras uses for learning purposes 2-not loaded-unknown

In [ ]:
word_to_index=imdb.get_word_index()

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
word_to_index['the']

1

In [ ]:
word_to_index['good']

49

### Reverse word_to_index mapping 

In [ ]:
index_to_word={index:word for(word,index) in word_to_index.items()}

In [ ]:
[index_to_word[i] for i in range(1,51)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more']

### Decode a Movie Review  
- i->3 accounts for the offsets,0-2-? use the key to return thr value

In [ ]:
' '.join([index_to_word.get(i-3,'?')for i in X_train[4]])

"? worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the ? and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life"

In [ ]:
y_train[3]

1

## Data Preparation
-review varies  
-same dimensions  
-prepare data for learning -number of words,truncate few pad 0
-pad_sequences

In [ ]:
words_per_review=200

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train=pad_sequences(X_train,maxlen=words_per_review)

In [ ]:
X_train.shape

(25000, 200)

In [ ]:
X_test=pad_sequences(X_test,maxlen=words_per_review)

In [ ]:
X_test.shape

(25000, 200)

## Split the data
test and validation data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,random_state=11,test_size=0.20)

In [ ]:
X_test.shape

(20000, 200)

In [ ]:
X_val.shape

(5000, 200)

## Creating the Neural Network

- Sequential model,then import the other layers

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
rnn=Sequential()

In [ ]:
rnn

In [ ]:
from tensorflow.keras.layers import Dense,LSTM,Embedding

## Embedding
-Adding an Embedding layer - In the CNN we used one hot encoding (converting MNIST into Categorical), each label then became a vector 9(0 0 0 0 0 0 0 0 0 1)  
- We can do the same for index values that represent words for example for 'the' the value data(1 0 0)  
- This becomes problematic with a large number of words  
- To solve this problem we begin with an Embedding layer to reduce dimensionality  
-This encodes each word in a more compact dense-vector representation.  
-The idea is to capture the words' context  
-RNN learns word relationships  
- There are very popular word embeddings- Word2Vec and Glove, and you can use them to load neural network and this will save you time, its also used to add basic word relationships to model smaller amount of training data, easy to improve model accuracy by building upon work that was done previously

In [ ]:
rnn.add(Embedding(input_dim=number_of_words,output_dim=128,input_length=words_per_review))

### Adding a LSTM layer
-Number of neurons- more neurons means network can remember more

In [ ]:
rnn.add(LSTM(units=128, dropout=0.2,recurrent_dropout=0.2))

### Adding a Dense Output layer
-To be able to reduce the output to one result-0 or 1-negative or positive  
-This is where we use the activation function; sigmoid function: best for binary classification

In [ ]:
rnn.add(Dense(units=1,activation='sigmoid'))

## Compile the model and display the summary
- Binary crossentropy as a loss function

In [ ]:
rnn.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


## Training the model

In [ ]:
rnn.fit(X_train,y_train,epochs=10,batch_size=32, validation_data=(X_test,y_test))

Epoch 1/10
782/782 [==============================] - 516s 656ms/step - loss: 0.4741 - accuracy: 0.7756 - val_loss: 0.4077 - val_accuracy: 0.8404
Epoch 2/10
782/782 [==============================] - 513s 656ms/step - loss: 0.2934 - accuracy: 0.8802 - val_loss: 0.3237 - val_accuracy: 0.8670
Epoch 3/10
782/782 [==============================] - 511s 653ms/step - loss: 0.1971 - accuracy: 0.9242 - val_loss: 0.3950 - val_accuracy: 0.8645
Epoch 4/10
782/782 [==============================] - 503s 643ms/step - loss: 0.1512 - accuracy: 0.9438 - val_loss: 0.3625 - val_accuracy: 0.8504
Epoch 5/10
782/782 [==============================] - 508s 650ms/step - loss: 0.2913 - accuracy: 0.8788 - val_loss: 0.3615 - val_accuracy: 0.8629
Epoch 6/10
782/782 [==============================] - 506s 648ms/step - loss: 0.1327 - accuracy: 0.9513 - val_loss: 0.3917 - val_accuracy: 0.8655
Epoch 7/10
782/782 [==============================] - 506s 647ms/step - loss: 0.0871 - accuracy: 0.9694 - val_loss: 0.4638 -

## Evaluating the Model

In [ ]:
results= rnn.evaluate(X_test,y_test)

In [ ]:
results

[0.5828183889389038, 0.8578000068664551]